In [1]:
import os

In [2]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

1366

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import json

In [6]:
for file in tqdm(list_file, desc='Progress', ncols=77):
    with open('./part/' + file, 'r', encoding='UTF-8') as f:
        html = f.read()
        
        # = = = = = = = = = = = = = = =
        
        soup = BeautifulSoup(html, 'lxml')
        
        # = = = = = = = = = = = = = = =
        
        html = etree.HTML(str(soup))

        title = html.xpath('//h1[@data-scroll-trigger-for-sticky-product-header]/text()')[0].strip()

        # = = = = = = = = = = = = = = =
        
        list_div = html.xpath('//div[contains(@class, "description-block")]')
        
        # = = = = = = = = = = = = = = =
        
        list_oe = [oe.strip() for oe in html.xpath('//div[@class="oe-number"]/descendant-or-self::*/text()')]

        while '' in list_oe:
            list_oe.remove('')

        list_oe = list(set(list_oe))

        oe = ';'.join(list_oe)

        # = = = = = = = = = = = = = = =
        
        part_code = [code.strip() for code in html.xpath('//div[@id="tabMatchingVehicles"]/@data-product-id')[0].strip()]
        
        # = = = = = = = = = = = = = = =
        
        key_value = json.loads(html.xpath('//script[@type="application/ld+json"]/text()')[0])
        
        # = = = = = = = = = = = = = = =
        
        df_temp = pd.DataFrame([{'序号': int(file.replace('.txt', '')),
                                 'Sku': key_value['@graph'][0]['sku'].strip(),
                                 'Title': title,
                                 'Manufacturer': key_value['@graph'][0]['manufacturer'].strip(),
                                 'Mpn': key_value['@graph'][0]['mpn'].strip(),
                                 'Price': key_value['@graph'][0]['offers'][0]['priceCurrency'].strip() + ' ' + key_value['@graph'][0]['offers'][0]['price'].strip(),
                                 'Kit': '',
                                 'Vehicle_1': '',
                                 'Vehicle_2': '',
                                 'OE': oe,
                                 'Pic': '',
                                 'Url': key_value['@graph'][0]['url'].strip(),
                                 'Src': key_value['@graph'][0]['image'].strip(),
                                 'Part_Code': part_code,
                                 'Make_Url': 'https://www.atp-autoteile.de/de/product/vehicles/' + part_code}])
        
        # = = = = = = = = = = = = = = =
        
        for div in list_div:
            if 'delivery-contents' in div.xpath('./@class')[0]:
                list_content = [content.strip() for content in div.xpath('./div[@class="description-content"]/text()')]

                while '' in list_content:
                    list_content.remove('')

                list_amount = [span.text.strip() for span in div.xpath('./div[@class="description-content"]/span[@class="delivery-content-count"]')]

                df_temp.loc[0, div.xpath('./div[@class="description-title"]/text()')[0].strip()] = '\n'.join([amount + ' ' + content for amount, content in zip(list_amount, list_content)])
            elif 'technical-info' in div.xpath('./@class')[0]:
                pass
            else:
                if len(div.xpath('./@style')) != 0:
                    if div.xpath('./@style')[0].strip() != 'display: none;':
                        print(key_value['@graph'][0]['url'].strip())
                        print(div.xpath('./@class')[0])
        
        # = = = = = = = = = = = = = = =
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =
        
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

Progress: 100%|██████████████████████████| 1366/1366 [00:58<00:00, 23.44it/s]


,序号,Sku,Title,Manufacturer,Mpn,Price,Kit,Vehicle,OE,Pic,Url,Src,Make_Page,Lieferumfang
0,1,40378624,BESTPRICE 40378624 – Satz Bremsbeläge +VWK +Zu...,BESTPRICE,40378624,EUR 87.77,,,95561236520;7L0698451H;7L0698451B;7L6698451A;9...,,https://www.atp-autoteile.de/de/product/409833...,https://www.atp-autoteile.de/media/product/200...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
1,2,40579708,ATEC 40579708 – Satz Bremsbeläge + Bremssättel...,ATEC,40579708,EUR 64.94,,,34116761237;34111163850;34111164500;3411116555...,,https://www.atp-autoteile.de/de/product/431665...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
2,3,34001991,BESTPRICE 34001991 – Satz Bremsbeläge + Bremss...,BESTPRICE,34001991,EUR 62.51,,,34116761237;34111163850;34111164500;3411116555...,,https://www.atp-autoteile.de/de/product/988398...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
3,4,31274813,BESTPRICE 31274813 – Satz Bremsbeläge +VWK Vor...,BESTPRICE,31274813,EUR 59.99,,,95561236520;7L0698451H;7L0698451B;7L6698451A;9...,,https://www.atp-autoteile.de/de/product/109350...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
4,5,40579659,ATEC 40579659 – Satz Bremsbeläge +VWK Vorder- ...,ATEC,40579659,EUR 58.09,,,95561236520;7L0698451H;7L0698451B;7L6698451A;9...,,https://www.atp-autoteile.de/de/product/431629...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,1362,40004092,BESTPRICE 40004092 – Bremsbeläge hinten (Satz),BESTPRICE,40004092,EUR 5.89,,,"95587471;95588498;95650872,;95624481,;95587472...",,https://www.atp-autoteile.de/de/product/373129...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Hinterachse\n1x ...
1362,1363,31528616,BESTPRICE 31528616 – Bremsbeläge hinten (Satz),BESTPRICE,31528616,EUR 5.84,,,",;34211159265;34211117981;34211150556",,https://www.atp-autoteile.de/de/product/372619...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Hinterachse\n1x ...
1363,1364,31528852,BESTPRICE 31528852 – Bremsbeläge vorne (Satz) ...,BESTPRICE,31528852,EUR 5.80,,,,,https://www.atp-autoteile.de/de/product/372777...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse inkl...
1364,1365,31478334,BESTPRICE 31478334 – Bremsbeläge hinten (Satz),BESTPRICE,31478334,EUR 5.80,,,",;0446632040;0446632030",,https://www.atp-autoteile.de/de/product/364739...,https://www.atp-autoteile.de/media/product/800...,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Hinterachse\n1x ...
